In [3]:
import pandas as pd
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset.drop('User ID', axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [7]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [8]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [9]:
dep=dataset[['Purchased']]
dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep,test_size=0.30,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
scx=StandardScaler()
X_train=scx.fit_transform(X_train)
X_test=scx.transform(X_test)

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={
    'weights':['uniform','distance'],
    'algorithm':['auto','ball_tree','brute','kd_tree'],
    'n_neighbors':[3,5,7]
}
grid=GridSearchCV(KNeighborsClassifier(),param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'brute', 'kd_tree'],
                         'n_neighbors': [3, 5, 7],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [13]:
Y_pred=grid.predict(X_test)
Y_pred

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 1], dtype=int64)

In [14]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,Y_pred)
print(cm)

[[73  6]
 [ 4 37]]


In [15]:
from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,Y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [16]:
from sklearn.metrics import f1_score,accuracy_score
f1_weighted=f1_score(Y_test,Y_pred, average='weighted')
print(f1_weighted)

accuracy=accuracy_score(Y_test,Y_pred)
print("Accuracy of the best model {} : {:.2f}".format(grid.best_params_, accuracy))

0.9171245421245421
Accuracy of the best model {'algorithm': 'auto', 'n_neighbors': 7, 'weights': 'uniform'} : 0.92


In [17]:
from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(Y_test, grid.predict_proba(X_test)[:,1])
print("roc_auc score for best parameter {}: {:.2f}".format(grid.best_params_,roc_score ))

roc_auc score for best parameter {'algorithm': 'auto', 'n_neighbors': 7, 'weights': 'uniform'}: 0.94


In [18]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.017148,0.012490,0.029964,0.002206,auto,3,uniform,"{'algorithm': 'auto', 'n_neighbors': 3, 'weigh...",0.892857,0.874254,0.841398,0.911105,0.910254,0.885974,0.026049,17
1,0.006658,0.001027,0.017584,0.003840,auto,3,distance,"{'algorithm': 'auto', 'n_neighbors': 3, 'weigh...",0.874254,0.874254,0.824293,0.893717,0.891667,0.871637,0.025075,21
2,0.008787,0.005714,0.033808,0.003679,auto,5,uniform,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",0.874254,0.874254,0.876643,0.929144,0.946153,0.900089,0.031147,5
3,0.009301,0.001585,0.021509,0.002021,auto,5,distance,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",0.855314,0.855314,0.859435,0.929144,0.946153,0.889072,0.040054,13
4,0.008188,0.003149,0.031154,0.003205,auto,7,uniform,"{'algorithm': 'auto', 'n_neighbors': 7, 'weigh...",0.874254,0.874254,0.859435,0.947015,0.964286,0.903849,0.042989,1
5,0.007736,0.001338,0.017943,0.003076,auto,7,distance,"{'algorithm': 'auto', 'n_neighbors': 7, 'weigh...",0.855314,0.874254,0.859435,0.929144,0.946153,0.892860,0.037496,9
6,0.005341,0.003416,0.026780,0.003715,ball_tree,3,uniform,"{'algorithm': 'ball_tree', 'n_neighbors': 3, '...",0.892857,0.874254,0.841398,0.911105,0.892857,0.882494,0.023623,19
7,0.008233,0.007818,0.016149,0.004942,ball_tree,3,distance,"{'algorithm': 'ball_tree', 'n_neighbors': 3, '...",0.874254,0.874254,0.824293,0.893717,0.874356,0.868175,0.023195,23
8,0.004206,0.004773,0.025252,0.004307,ball_tree,5,uniform,"{'algorithm': 'ball_tree', 'n_neighbors': 5, '...",0.874254,0.874254,0.876643,0.929144,0.946153,0.900089,0.031147,5
9,0.004447,0.004840,0.018260,0.003829,ball_tree,5,distance,"{'algorithm': 'ball_tree', 'n_neighbors': 5, '...",0.855314,0.855314,0.859435,0.929144,0.946153,0.889072,0.040054,13
